In [ ]:
import scanpy as sc
import pandas as pd
from os import path
from os.path import join as pj 
import math
import numpy as np
import torch as th
import pandas as pd
import os
from torch import nn, autograd 
from sklearn.mixture import BayesianGaussianMixture
from modules import models, utils
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import silhouette_score
from sklearn import metrics
from numpy import *
from sklearn.cluster import KMeans

In [ ]:

file_z_pca = './data/z1/pca_x1.csv'
path_label = './data/z1/label_seurat/label.csv'
adatapca = sc.read(file_z_pca)
sc.pp.normalize_total(adatapca, target_sum=1e4)
sc.pp.neighbors(adatapca, n_neighbors=30,n_pcs = 32)

results = {}
weightrange = arange(0.01, 5, 1)
# for cn_components in np.array([12,14,16, 17, 18]):
for n_neighbor in np.array([15, 30, 50, 100]):
    for n_pc in np.array([5, 10,15,20,25,30,32]):
        for resolutions in np.array([0.01,0.1,1,10,100]):
        # print('%s-%s'%(cn_components, cweight_concentration_prior))
            sc.pp.neighbors(adatapca, n_neighbors=n_neighbor,n_pcs = n_pc)
            sc.tl.louvain(adatapca,resolution=resolutions)
            label_p = adatapca.obs['louvain']
            label_p.to_csv('./data/z1/label_louvain.csv')
            sc.tl.leiden(adatapca, resolution=resolutions)
            label_ppcalei = adatapca.obs['leiden']
            label_ppcalei.to_csv('./data/z1/label_leiden.csv')
            path_label2 = './data/z1/label_louvain.csv'
            path_label3 = './data/z1/label_leiden.csv'
            label = utils.load_csv(path_label)
            label_tlist = utils.transpose_list(label)[1][1:]
            # label_true = np.array(label_tlist)
            label_predictlo = utils.load_csv(path_label2)
            label_plistlo = utils.transpose_list(label_predictlo)[1][1:]
            label_predictle = utils.load_csv(path_label3)
            label_plistle = utils.transpose_list(label_predictle)[1][1:]
            # label_plist = utils.transpose_list(label_pre_ten)[0]
            z_list = utils.load_csv(file_z_pca)
            z_np = np.array(z_list, dtype=np.float64)
            arilo = adjusted_rand_score(label_tlist, label_plistlo) #l1 kpca20
            # print("arilo:", arilo) 
            nmilo = normalized_mutual_info_score(label_tlist, label_plistlo)
            # print("nmilo:", nmilo)
            sclo = silhouette_score(z_np, label_plistlo)
            # print("sclo", sclo) 
            arile = adjusted_rand_score(label_tlist, label_plistle) #l1 kpca20
            # print("arile:", arile) 
            nmile = normalized_mutual_info_score(label_tlist, label_plistle)
            # print("nmile:", nmile)
            scle = silhouette_score(z_np, label_plistle)
            # print("scle", scle) 
            print("n_neighbor", n_neighbor,"n_pc",n_pc,  "resolution",resolutions,"arilo:", arilo, "nmilo:", nmilo, "sclo", sclo,"arile:", arile,"nmile:", nmile,"scle", scle)